In [240]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import pickle

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score, r2_score, mean_squared_log_error
from pandas.plotting import scatter_matrix
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

from sklearn.feature_selection import SelectKBest, f_regression, chi2, f_classif

In [241]:
def class_feature_selection_transformation(X, y, target, ordinal_feature, route):
    
    def train_num_feature_selection(X_num):
        # feature selection on numerical data
        k_num = round(len(X_num.columns) / 2)
        fs = SelectKBest(f_classif, k_num)
        fs.fit(X_num, y) # save!!
        X_num_fs = fs.transform(X_num)
        return (X_num_fs, fs)
        
    def train_cat_feature_selection(X_cat, X_cat_enc):
        # feature selection on categorical data
        k_cat = round(len(X_cat.columns) / 2)
        fs = SelectKBest(chi2, k_cat)
        fs.fit(X_cat_enc, y) # save!!
        X_cat_fs = fs.transform(X_cat_enc)
        X_cat_enc = pd.DataFrame(X_cat_fs)
        return (X_cat_enc, fs)
    
    def predict_num_feature_selection(X_num):
        with open('fs_values.pkl', 'rb') as file:
            selected_num_features, dummy = pickle.load(file)[0:2]
            
        X_num_fs = selected_num_features.transform(X_num)
        X_num = pd.DataFrame(X_num_fs)
        
        return X_num
    
    def predict_cat_feature_selection(X_cat_enc):
        with open('fs_values.pkl', 'rb') as file:
            dummy, selected_cat_features = pickle.load(file)[0:2]
            
        X_cat_fs = selected_cat_features.transform(X_cat_enc)
        X_cat_enc = pd.DataFrame(X_cat_fs)
        
        return X_cat_enc
    
    # split features
    num_features = []
    cat_features = []
    X_num = pd.DataFrame()
    X_cat = pd.DataFrame()
    
    for feature in X:
        if X[feature].dtypes == np.int or X[feature].dtypes == np.float:
            num_features.append(feature)
        else:
            cat_features.append(feature)
    
    # impute using only numerical features
    if num_features:
        X = X.reset_index(drop = True)
        imp = IterativeImputer(max_iter = 10, random_state = 42)
        imp.fit(X[num_features])
        X[num_features] = imp.transform(X[num_features])
        X_num = X.drop(cat_features, axis = 1)
    
    # impute using only categorical features
    if cat_features and not all(elem == target for elem in cat_features):
        imp = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
        X[cat_features] = imp.fit_transform(X[cat_features].astype(str))
        X_cat = X.drop(num_features, axis = 1)
        
    # get column count
    if not X_num.empty:
        num_shape = X_num.shape[1]
    else:
        num_shape = 0
    
    if not X_cat.empty:
        cat_shape = X_cat.shape[1]
    else:
        cat_shape = 0
    
    # drop target
    if target in X_num:
        X_num = X_num.drop(target, axis = 1)
    if target in X_cat:
        X_cat = X_cat.drop(target, axis = 1)
    
    # feature selection on num_features
    if route == '/train':
        if num_features and not num_shape <= 10:
            X_num, selected_num_features = train_num_feature_selection(X_num)
        else:
            selected_num_features = None
    if route == '/predict':
        if num_features and not num_shape <= 10:
            X_num = predict_num_feature_selection(X_num)
    
    # encode ordinal features (dummy variables)
    if ordinal_feature is not None:
        ord_data = [ordinal_feature]
        X_num = pd.get_dummies(X_num, columns = ord_data, drop_first = True)
    
    if cat_features:
        enc = OrdinalEncoder()
        enc.fit(X_cat)
        X_cat_enc = enc.transform(X_cat)
    
    # feature selection on cat_features
    if route == '/train':
        if cat_features and not cat_shape <= 10:
            X_cat_enc, selected_cat_features = train_cat_feature_selection(X_cat, X_cat_enc)
        else:
            selected_cat_features = None
    if route == '/predict':
        if cat_features and not cat_shape <= 10:
            X_cat_enc = predict_cat_feature_selection(X_cat_enc)
    
    # concatenate numerical and categorical features
    if num_features and cat_features:
        df_cat = pd.DataFrame(X_cat_enc, index = list(range(len(X.index))))
        df_num = pd.DataFrame(X_num, index = list(range(len(X.index))))
        X = pd.concat([df_cat, df_num], axis = 1, sort = False)
    elif cat_features:
        X = pd.DataFrame(X_cat_enc)
    elif num_features:
        X = pd.DataFrame(X_num)
    
    if route == '/train':
        # serialize feature selection values
        fs_values = [selected_num_features, selected_cat_features, ordinal_feature, target]
        with open('fs_values.pkl', 'wb') as file:
            pickle.dump(fs_values, file)
            
    return X

In [242]:
def predict_randomforestclass(X, y, target):
    if target in X:
        X = X.drop(target, axis = 1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
    
    rf = RandomForestClassifier(n_estimators = 100, max_depth = None) 
    rf.fit(X_train, y_train)
    
    # quantifying the quality of prediction
    y_predict = rf.predict(X_test)
    acc_score = abs(accuracy_score(y_test, y_predict))
    
    model = (rf, acc_score)
    return model

In [243]:
def predict_logisticregress(X, y, target):
    if target in X:
        X = X.drop(target, axis = 1)
    X_scaled = preprocessing.scale(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state = 42)
    
    log_reg = LogisticRegression(random_state = 42) 
    log_reg.fit(X_train, y_train)
    
    # quantifying the quality of prediction
    y_predict = log_reg.predict(X_test)
    acc_score = abs(accuracy_score(y_test, y_predict))
    
    model = (log_reg, acc_score)
    return model

In [244]:
def train():
    def predict_class(feature_engineering, y, target):
        rf, rf_acc = predict_randomforestclass(feature_engineering, y, target)
        log_reg, log_acc = predict_logisticregress(feature_engineering, y, target)
        
        best_score = max([rf_acc, log_acc])

        # save best model on disk
        if best_score == rf_acc:
            with open('model.pkl', 'wb') as file:
                pickle.dump(rf, file)
                model = 'RandomForestClassifier: '
                return (model, rf_acc)
        elif best_score == log_acc:
            with open('model.pkl', 'wb') as file:
                pickle.dump(log_reg, file)
                model = 'LinearRegression: '
                return (model, log_acc)
    
    train_data = pd.read_csv(os.path.expanduser("~/Desktop/Projects/api/data/titanic_train.csv"), index_col = [0])
    
    # select features and target variable
    target = 'Survived'
    ordinal_feature = None
    route = '/train'
    
    features = list(train_data)
    X = train_data[features]
    y = train_data[target]
    
    # only for iris
    # X = X.drop('Id', axis = 1)

    feature_engineering = class_feature_selection_transformation(X, y, target, ordinal_feature, route)
    model, best_score = predict_class(feature_engineering, y, target)
    prediction = model + 'Accuracy Score: ' + str(best_score) 
    
    return prediction

In [245]:
train()

'RandomForestClassifier: Accuracy Score: 0.852017937219731'

In [246]:
def predict():
    
    test_data = pd.read_csv(os.path.expanduser("~/Desktop/Projects/api/data/titanic_test.csv"), index_col = [0])
    route = '/predict'
    
    with open('fs_values.pkl', 'rb') as file:
        ordinal_feature, target = pickle.load(file)[2:]
    with open('model.pkl', 'rb') as file:
        model = pickle.load(file)
        
    # test_data = test_data.reset_index(drop = True)
    
    # nur bei iris
    # test_data = test_data.drop('Species', axis = 1)
    
    feature_engineering = class_feature_selection_transformation(test_data, None, None, ordinal_feature, route)
    y_predict = model.predict(feature_engineering)
    
    df_features = pd.DataFrame(test_data)
    df_prediction = pd.DataFrame({target: y_predict})
    output = pd.concat([df_features, df_prediction], axis = 1, sort = False)
    output.to_csv('prediction.csv', index = False)
    output.describe()

    return output

In [247]:
predict()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1305,3.0,"Spector, Mr. Woolf",male,NaN,0.0,0.0,A.5. 3236,8.0500,NaN,S,NaN
1306,1.0,"Oliva y Ocana, Dona. Fermina",female,39.0,0.0,0.0,PC 17758,108.9000,C105,C,NaN
1307,3.0,"Saether, Mr. Simon Sivertsen",male,38.5,0.0,0.0,SOTON/O.Q. 3101262,7.2500,NaN,S,NaN
1308,3.0,"Ware, Mr. Frederick",male,NaN,0.0,0.0,359309,8.0500,NaN,S,NaN
